<a id="plan"></a>



# **Predicting a song's populatiry on Spotify with Random Forrests<br><br>**
 
  

      **Justine Ferret, Arthur Guyart et Yanis Martinet**


### Imports

In [13]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# base modules
import os
import sys

import copy
import logging

# custom module
from emlyon_module.imports import *
from emlyon_module.structured import *

# for manipulating data
from pandas_summary import DataFrameSummary
!pip install dill
import dill

# for Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance
from scipy.cluster import hierarchy

# for visualization
from IPython.display import display
from matplotlib import pyplot as plt
!pip install -U plotnine
from plotnine import ggplot, aes
from plotnine.stats import stat_smooth
from pdpbox import pdp
# plotly
# seaborn
# altair

ImportError: cannot import name '_ccallback_c'

# **I) Data Inspection**

# a) Import the data

[Back to top](#plan)

In [4]:
path_to_repo = os.path.dirname(os.getcwd())

path = os.path.join(path_to_repo, "data", "")

path_to_datasets = os.path.join(path, 'data.csv')

df_raw = pd.read_csv(path_to_datasets, low_memory = False, parse_dates = ["release_date"])

NameError: name 'pd' is not defined

<a id="look_data"></a>

# b) First look on the data

First let's look the type of data and column name

In [ ]:
df_raw.dtypes

In [ ]:
df_raw

Let's have a quick look on the data repartition over each music characteristics

In [ ]:
df_raw.hist(figsize=(20, 20), color='Blue')
plt.show()

These graph shows us the unbalanced data which will need to be retreat through intervals. Therefore the unbalanced identied data are on the acousticsness and intrumentalness.

# c) Correlation Inspection

Too have a first sigh of the possible correlation over the popularity and our dataset, we must sort the descending correlation from all our columns

In [ ]:
corr = np.abs(df_raw.corr())
series = np.abs(corr['popularity']).sort_values(ascending=False)
for i, row in enumerate(series):
    if 0.1 <= row < 1:
      print(f'{series.index[i]:17} --> {row: .2f} (abs)')

The year, acousticness and loudness appears to be the three main correlators to the popularity. For now artists do not seems correlate to popularity, this might result from the format of the artist column in the dataset.

# d) Inspection of our predicted variable - Popularity

[Back to top](#plan)

The variable we want to predict is the popularity of a song, based on its characteristics, artist and date of release. So let's have a look on the popularity data
[texte du lien](https://)

In [ ]:
df_raw.popularity.describe()

In [ ]:
df_raw[df_raw['popularity'] > 90]

The popularity rating is a numerical value between 0 and 100 attributed to each song, 100 meaning that a song is very popular. Out of 173,292 total songs, only 31 have a popularity rating of 90 or above.

# **II) Data Preprocessing**

# A) **Preprocessing of Artists Name**

As we can see, the elements in the 'artists' column are actually sublists. For example on the row 174,384, the song has 3 different artists. In order to process the fact that a song could have multiple.
*We will add a column for each secondary artist on a song so that the model can process each artist individualy instead of processing the list as a single element.*

In [ ]:
for i, l in enumerate(df_raw["artists"]):
    print("list",i,"is",type(l))

The sublists aren't event seen as lists by Pandas in the first place but rather as strings.

In [ ]:
df_raw["artists"] = df_raw["artists"].

 Deleting a part of the artists

*As the Artist columns is equal to the list of all the artist who engaged within the song, we must split these list and evaluate the great number or artist to keep.*

In [ ]:
max(len(l) for l in df_raw['artists'])

The song with the most recording artists on this dataset has 28 different people on it. A simpler way to deal with that might be to cut the rows with too many artists on a single song so that we won't create too many columns that won't be used.

In [ ]:
size = []
for i in df_raw["artists"]:
    size.append(len(i))

In [ ]:
for n in range(1,29):
    print([n, sum(i >= n for i in size), "{:.4%}".format(sum(i >= n for i in size)/len(size))])

Only 1,097 rows (0.6% of the dataset) have 6 or more artists, we'll drop these rows so that only 4 secondary artists columns will have to be created.

In [ ]:
df_raw['#artists'] = size

In [ ]:
df_raw = df_raw[df_raw['#artists'] <= 5]
df_raw = df_raw.reset_index(drop=True)

In [ ]:
artists = pd.DataFrame.from_records(df_raw['artists'])
artists

#  Artists Column Subdivision

*We choose to split the list of the 5 artist into 5 different column to identify each of them. *


In [ ]:
df_raw["main_artist"] = artists[0]
df_raw["artist2"] = artists[1]
df_raw["artist3"] = artists[2]
df_raw["artist4"] = artists[3]
df_raw["artist5"] = artists[4]

df_raw = df_raw.drop(columns=['artists'])

In [5]:
df_raw

NameError: name 'df_raw' is not defined

# **Creation of Artist Popularity Score** 

*To get a better model we choose to replace the artist name by a score of their popularity*

Therefore, we  created a fonction to attribute a popularity score to the  Artists of each music. For this we will take the mean popularity of the music with the Artist in it and then do the sum of artist popularity for the artists within the music.


In [ ]:
df_raw['mean_art_popularity'] = df_raw.groupby('main_artist')['popularity'].transform('mean')

In [ ]:
df_raw['mean_art2_popularity'] = df_raw.groupby('artist2')['popularity'].transform('mean')

In [ ]:
df_raw['mean_art3_popularity'] = df_raw.groupby('artist3')['popularity'].transform('mean')

In [ ]:
df_raw['mean_art4_popularity'] = df_raw.groupby('artist4')['popularity'].transform('mean')

In [ ]:
df_raw['mean_art5_popularity'] = df_raw.groupby('artist5')['popularity'].transform('mean')

In [ ]:
df_raw['All_Artists_pop_sum'] = df_raw[['mean_art5_popularity','mean_art4_popularity','mean_art3_popularity','mean_art2_popularity','mean_art_popularity']].mean(axis=1)

In [ ]:
df_raw = df_raw.drop(labels=['mean_art5_popularity','mean_art4_popularity','mean_art3_popularity','mean_art2_popularity','mean_art_popularity'], axis=1)

In [ ]:
df_raw

<a id="look_columns"></a>

# **B) Look at the musics characteristics  columns**

[Back to top](#plan)

To continue, we must first focus on the unbalanced data identified previously. 

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
display_all(df_raw.describe(include='all', datetime_is_numeric=True).T)

**Instrumentalness and Acousticness Classification retreatment**

Acousticness has been identified as a major correlation for the popularity and represent unbalanced data so we must investigate on it and see if we can improve the data. Also as seen previously intrumentalness and acousticness data do not have any homegeneity.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.distplot(df_raw['acousticness'], kde=False, bins=10)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.distplot(df_raw['instrumentalness'], kde=False, bins=10)
plt.show()

Songs seem to be either acoustic or not, they are likely not a combination of both. We will then create categories (acoustic, mix, non-acoustic). Similarly, we'll do the same thing on the instrumentalness and loudness feature.

In [ ]:
df_raw['instrumentalness'] = df_raw['instrumentalness'].map((lambda x: 1 if x < 0.1 else (3 if x > 0.8 else 2)))

In [ ]:
df_raw['acousticness'] = df_raw['acousticness'].map((lambda x: 1 if x < 0.1 else (3 if x > 0.9 else 2)))

In [ ]:
df_raw

## C) Evaluation of the date information within our Dataset**

In [ ]:
#df_raw['year'] = pd.DatetimeIndex(df_raw['release_date']).year

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
ax = df_raw.groupby('year')['popularity'].mean().plot()
ax.set_title('Mean Popularity per year', weight='bold')
ax.set_ylabel('Mean Popularity')
ax.set_xlabel('Year')
ax.set_xticks(range(1920, 2021, 5))
plt.show()

We will wipe out the 2021 song because there is not enough popularity data on this year and so to avoid any outlier influence we keep only the rest of the dataset. We do the same for the song before 1955 as their popularity data appears not relevant. ( Their low popularity could be explained by the lack of diffusion on these time )

In [ ]:
df_raw = df_raw[df_raw['year'] < 2021]
df_raw = df_raw.reset_index(drop=True)

In [ ]:
df_raw = df_raw[df_raw['year'] > 1955]
df_raw = df_raw.reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
ax = df_raw.groupby('year')['popularity'].mean().plot()
ax.set_title('Mean Popularity per year', weight='bold')
ax.set_ylabel('Mean Popularity')
ax.set_xlabel('Year')
ax.set_xticks(range(1956, 2020, 5))
plt.show()

It could also be interesting to investigate on the seasonality of the popularity. Let's create a column with the month and see if song are more popular among a season 

In [ ]:
df_raw['month'] = pd.DatetimeIndex(df_raw['release_date']).month

In [ ]:
sns.scatterplot(x="month", y="popularity", data=df_raw, alpha=0.01, color='blue')

It seems that there is no correlation between the month and the popularity level

<a id="output_variable"></a>

# **D) Convert MS duration into minute duration**

In the current dataset all the music times are displayed within ms, which could badly influence our model so we choose to convert it into minutes

In [ ]:
df_raw["duration_min"] = df_raw["duration_ms"]/(1000 * 60)
df_raw["duration_min"] = df_raw["duration_min"].astype(int)
df_raw = df_raw.drop(labels=['duration_ms'], axis=1)

In [ ]:
sns.scatterplot(x="duration_min", y="popularity", data=df_raw, alpha=0.03, color='blue')

The above graph show us that the popularity appears correlate with the duration. Indeed, the longer songs seems to be less popular ( >15min), also short songs seems to be less popular ( <2min30 ). So we will create three different classification to represent the duration differences


In [ ]:
df_raw['duration_min'] = df_raw['duration_min'].map((lambda x: 1 if x < 2.30 else (3 if x > 15 else 2)))

# **E) Language of the song**

We suppose that the song language influence its popularity. Indeed, our dataset include song from all over the world and a Russian song will obviously be less popular than an English song that could be more oftenly become a worldwide hit. 
Therefor we will try to identify the language of everysong from the music name and see if it correlates with the popularity. If that the case, we will keep this data to improve the dataset 

In [ ]:
!pip install langdetect
import langdetect
from langdetect import detect

In [ ]:
for index, row in df_raw['name'].iteritems():
  try:
    lang = detect(str(row)) #detecting each row
    df_raw.loc[index, 'language'] = lang
    print(index)
  except:
    lang = 'na'
    df_raw.loc[index, 'language'] = lang
    print(lang)

In [ ]:
#df_raw['languageindex'] = pd.DataFrame(df_raw, columns= ['language'])

nombrelangue = df_raw.pivot_table(index=['language'], aggfunc='size')
print(df_raw)

Now we have for each country an unique value that can be use within our model later on

In [ ]:
corr = np.abs(df_raw.corr())
series = np.abs(corr['popularity']).sort_values(ascending=False)
for i, row in enumerate(series):
    if 0.1 <= row < 1:
      print(f'{series.index[i]:17} --> {row: .2f} (abs)')

After the preprocessing we observ that the correlation between the popularity and the data set have changed. The Artist popularity score and the year appears now as the main correlators.

# **F) Dropping of Useless Columns**

In [ ]:
df_raw

In [ ]:
df_raw = df_raw.drop(labels=['id','name','year','artist2','artist3','artist4','artist5','main_artist'], axis=1)

<a id="convert_date"></a>

# $\bullet$ First  Model 

[Back to top](#plan)

In [ ]:
df_backup = copy.deepcopy(df_raw)

In [ ]:
df_raw = df_backup

In [ ]:
add_datepart(df_raw, 'release_date')

In [ ]:

df, y, nas = proc_df(df_raw, 'popularity')

In [ ]:
def rmse(y_gold, y_pred): 
    return math.sqrt(((y_gold - y_pred)**2).mean())

def print_score(m, X_train, y_train, X_valid, y_valid):
    print('RMSE on train set: {:.4f}'.format(rmse(m.predict(X_train), y_train)))
    print('RMSE on valid set: {:.4f}'.format(rmse(m.predict(X_valid), y_valid)))
    print('R^2 on train set: {:.4f}'.format(m.score(X_train, y_train)))
    print('R^2 on valid set: {:.4f}'.format(m.score(X_valid, y_valid)))
    if hasattr(m, 'oob_score_'): print('R^2 on oob set: {:.4f}'.format(m.oob_score_))
    return

In [ ]:
def split_vals(df, n): 
    return df[:n].copy(), df[n:].copy()

In [ ]:
n_total = len(df)
n_valid = 25000
n_train = n_total - n_valid
n_small = 25000

print('full number of data points : {}'.format(n_total))
print('number of validation data points : {}'.format(n_valid))
print('number of training data points : {}'.format(n_train))
print('number of subsampled training points : {}'.format(n_small))

In [ ]:
df_raw = df_raw.sample(frac=1).reset_index(drop=True)
df, y, nas = proc_df(df_raw, 'popularity')

X_train, X_valid = split_vals(df, n_train)
y_train, y_valid = split_vals(y, n_train)
X_small, _ = split_vals(X_train, n_small)
y_small, _ = split_vals(y_train, n_small)

print('Number of full training data points: X = {}, y = {}'.format(X_train.shape, y_train.shape))
print('Number of small training data points: X = {}, y = {}'.format(X_small.shape, y_small.shape))
print('Number of validation data points: X = {}, y = {}'.format(X_valid.shape, y_valid.shape))

In [ ]:
base_model = RandomForestRegressor(n_jobs = -1, random_state = 42)

%time base_model.fit(X_train, y_train)
print_score(base_model, X_train, y_train, X_valid, y_valid)

<a id="convert_date"></a>

### $\bullet$ Save preprocessed data

[Back to top](#plan)

But let's save this file for now, since it's already in a format which can be stored and accessed efficiently.

In [ ]:
path_to_tmp = os.path.join(path_to_repo, "data", "tmp")

In [ ]:
path_to_tmp

In [ ]:
os.makedirs(path_to_tmp, exist_ok = True)

In [ ]:
path_to_data_raw = os.path.join(path_to_tmp, 'spotify-raw')

In [ ]:
path_to_data_raw

In [ ]:
df.to_feather(path_to_data_raw)

In [ ]:
df.to_csv(path_to_data_raw + '.csv', index = False)

<a id="numericalize_data"></a>

# 3. Random Forests

[Back to top](#plan)

### $\bullet$ Out-of-bag (OOB) score

The OOB score is a good indicator to have a clearer view of how the model performs.

[Back to top](#plan)

In [ ]:
model = RandomForestRegressor(
    n_jobs = -1, 
    random_state = 42,
    
    bootstrap = True,
    oob_score = True, # default = False, 
    max_samples = None,
)

%time model.fit(X_train, y_train)
print_score(model, X_train, y_train, X_valid, y_valid)

The model is slightly overfitting, we should explore solutions to prevent that from happening.

<a id='data_sampling'></a>

### $\bullet$ Data subsampling

[Back to top](#plan)

It turns out that one of the easiest ways to avoid over-fitting is also one of the best ways to speed up analysis: *subsampling*.

In [ ]:
model = RandomForestRegressor(
    n_jobs = -1, 
    random_state = 42,
    
    bootstrap = True,
    oob_score = True, # default = False, 
    max_samples = 0.15, # default = None,
)

%time model.fit(X_train, y_train)
print_score(model, X_train, y_train, X_valid, y_valid)

<a id='feature_sampling'></a>

### $\bullet$ Feature subsampling

[Back to top](#plan)

In [ ]:
model = RandomForestRegressor(
    min_samples_split = 10,
    max_features = 0.5, # default = 'auto'/None
    n_jobs = -1, 
    random_state = 42,
    
    bootstrap = True,
    oob_score = True, # default = False, 
    max_samples = 0.75, # default = None,
)

%time model.fit(X_train, y_train)
print_score(model, X_train, y_train, X_valid, y_valid)

<a id='general_bagging'></a>

### $\bullet$ Cross validation

[Back to top](#plan)

In [ ]:
# native Random Forest
model = RandomForestRegressor(
    min_samples_split = 2,
    max_features = None, # default = 'auto'/None
    n_jobs = -1, 
    random_state = 42,
    
    bootstrap = True,
    oob_score = True, # default = False, 
    max_samples = None,
)

%time model.fit(X_train, y_train)
print_score(model, X_train, y_train, X_valid, y_valid)

In [ ]:
# default is cv = 5
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_small, y_small, cv = 8, n_jobs = -1)

print(scores)
print("r2 is {:.3f} with a standard deviation of {:.3f}".format(scores.mean(), scores.std()))

- Change the score formula (ex: switch from $R^2$ to Root Mean Squared Error)

In [ ]:
scores = cross_val_score(model, X_small, y_small, cv = 8, scoring = 'r2', n_jobs = -1)

print(scores)
print("r2 is {:.3f} with a standard deviation of {:.3f}".format(scores.mean(), scores.std()))

In [ ]:
scores = cross_val_score(model, X_small, y_small, cv = 8, scoring = 'neg_root_mean_squared_error', n_jobs = -1)

print(scores)
print("RMSE is {:.3f} with a standard deviation of {:.3f}".format(-scores.mean(), scores.std()))

In [ ]:
# list of possible keys for scoring
sorted(sklearn.metrics.SCORERS.keys())

<a id='hyperparameter_tuning'></a>

### $\bullet$ Hyperparameter tuning for final model selection

[Back to top](#plan)

In [ ]:
model = RandomForestRegressor(
    n_jobs = -1, 
    random_state = 42,
    
    bootstrap = True,
    oob_score = True, # default = False, 
    max_samples = None,
)

%time model.fit(X_small, y_small)
print_score(model, X_small, y_small, X_valid, y_valid)

In [ ]:
param_grid = [
    
    {'n_estimators': [30],
     'max_depth': [5,10,15],
     'min_samples_split': [2,5,10,15],
     'min_samples_leaf':[1,5,10]
    },
 ]

model = RandomForestRegressor(
    random_state = 42,
    bootstrap = True,
    oob_score = True,
)

In [ ]:
from sklearn.model_selection import GridSearchCV

tuned_model = GridSearchCV(
    estimator = model, 
    param_grid = param_grid, 
    scoring = None, # uses estimator's default score method
    n_jobs = -1, 
    refit = True, # keep a fitted version of the overall best model
    cv = 5, 
    return_train_score = True,
    verbose = 5,
)

In [ ]:
%time tuned_model.fit(X_small, y_small)

print('Train R^2 Score : %.3f'%tuned_model.best_estimator_.score(X_train, y_train))
print('Test R^2 Score : %.3f'%tuned_model.best_estimator_.score(X_valid, y_valid))
print('Best R^2 Score Through Grid Search : %.3f'%tuned_model.best_score_)
print('Best Parameters : ',tuned_model.best_params_)

In [ ]:
best_model = tuned_model.best_estimator_
best_model

In [ ]:
print_score(best_model, X_small, y_small, X_valid, y_valid)

Though this model doesn't seem to be overfitting, the scores ares a bit worse that what we could find while we were exploring solutions. Let's check another parameter grid.

In [ ]:
param_grid = [
    {'n_estimators': [30],
     'max_depth': [10, 20, None],
     'min_samples_split': [2, 10, 50],
     'max_features': [0.25, 0.5, 0.75],
     'max_samples': [0.2, 0.5, 0.75],
    },
 ]

model = RandomForestRegressor(
    random_state = 42,
    oob_score = True,
    bootstrap = True,
)

In [ ]:
tuned_model = GridSearchCV(
    estimator = model, 
    param_grid = param_grid, 
    scoring = None, # uses estimator's default score method
    n_jobs = -1, 
    refit = True, # keep a fitted version of the overall best model
    cv = 5, 
    return_train_score = True,
    verbose = 5,
)

In [ ]:
%time tuned_model.fit(X_small, y_small)

In [ ]:
best_model = tuned_model.best_estimator_
best_model

In [ ]:
print_score(best_model, X_small, y_small, X_valid, y_valid)

In [ ]:
# randomized splitting strategy
cv = KFold(n_splits = 5, shuffle = True, random_state = 42)

scores = cross_val_score(best_model, X_small, y_small, cv = cv, n_jobs = -1)

print(scores)
print("r2 is {:.3f} with a standard deviation of {:.3f}".format(scores.mean(), scores.std()))

In [ ]:
model = RandomForestRegressor(max_depth=10, max_features=0.75, max_samples=0.75,
                      n_estimators=30, oob_score=True, random_state=42
)

%time model.fit(X_train, y_train)
print_score(model, X_train, y_train, X_valid, y_valid)

# 4. Classification, and differences with Regression

[Back to top](#plan)

We shall change our target variable from log sale prices to a set of _categorical sale prices_. Then we will use the `RandomForestClassifier` model to compute prediction. Classifiers are typically used in order to predict one class among a finite set of possibilities, however these models actually generate a *probability estimate* over the full set of possibilities. The resulting prediction is then simply taken as the most likely class given this output probability estimate.


Let us introduce the use of classifiers on out dataset, with the continuous variable *SalePrice* converted into a categorical one.

<a id='categorical_target'></a>

### $\bullet$ Categorical target

[Back to top](#plan)

We shall change our target variable from log sale prices to a set of _categorical sale prices_.

In [ ]:
pd.Series(y_train).plot(kind = 'density', bw_method = 0.05);

# alternatively
# sns.set_style('whitegrid')
# sns.kdeplot(np.array(y_train), bw_method = 0.05)

We will recast the sale Price prediction problem as a _binary classification_ problem:

In [ ]:
y[:10]

In [ ]:
# intervals are taken along the median (10.165):
# ]8.463, 10.165]
# ]10.165, 11.864]
intervals_2 = pd.cut(y, bins = 2)

In [ ]:
intervals_2[:10].tolist()

In [ ]:
type(intervals_2[0])

In [ ]:
label_encoder = LabelEncoder()

y_cat_2 = label_encoder.fit_transform(intervals_2)

In [ ]:
y_cat_2[:10]

In [ ]:
label_encoder.classes_.tolist()

In [ ]:
y_cat_2_train, y_cat_2_valid = split_vals(y_cat_2, n_train)
y_cat_2_small, _ = split_vals(y_cat_2_train, n_small)

y_cat_2_train.shape, y_cat_2_valid.shape, y_cat_2_small.shape

In [ ]:
pd.Series(y_cat_2_train).plot(kind = 'density', bw_method = 0.05);

# alternatively
# sns.set_style('whitegrid')
# sns.kdeplot(np.array(y_train), bw_method = 0.05)

<a id='random_forest_classifier'></a>

The calibrated version of the Random Forest Classifier has more balanced precision and recall scores. These scores are moreover more stable with respect to the chosen threshold, as shifting the classification threshold from 0.4 to 0.6 induces this time a reduced 9% precision jump and 18% recall jump.

#### Calibration for probability interpretation as confidence levels

We use here calibration as a mean to reduce variance in the resulting deterministic classifiers when shifting the decision threshold. Originally, this method was designed in order to get probabilistic models whose _probability estimates truly correspond to confidence levels_, in the sense that the region of input points that were assigned a probability of $p$ to be of a given label should form a region where the empirical ratio of points with this label is $p$.<br>
For more details see the [scikit-learn documentation](https://scikit-learn.org/stable/modules/calibration.html#calibration)

### $\bullet$ Multiclass classification

[Back to top](#plan)


Let us replicate the process followed for binary classification to the case of 5-fold classification

#### Data categorization

In [ ]:
calibrated_classifier = CalibratedClassifierCV(
    base_estimator = classifier,
    method = 'sigmoid', # 'sigmoid' or 'isotonic'
    ensemble = True,
    cv = 8, # int of 'prefit'
    n_jobs = -1,
)

In [ ]:
probas = calibrated_classifier.predict_proba(X_valid)[:, 1]

precision_recall_curve_with_threshold(y_cat_2_valid, probas, threshold = 0.5)

In [ ]:
intervals_5 = pd.cut(y, bins = 5)
intervals_5[:10].tolist()

In [ ]:
label_encoder = LabelEncoder()

y_cat_5 = label_encoder.fit_transform(intervals_5)
y_cat_5[:10]

In [ ]:
label_encoder.classes_.tolist()

In [ ]:
y_cat_5_train, y_cat_5_valid = split_vals(y_cat_5, n_train)
y_cat_5_small, _ = split_vals(y_cat_5, n_small)

y_cat_5_train.shape, y_cat_5_valid.shape, y_cat_5_small.shape

#### Model training

In [ ]:
classifier = RandomForestClassifier(
    n_estimators = 20, 
    class_weight = 'balanced', # classifier specific
    criterion = 'gini',  # classifier specific
    max_depth = 3, 
    min_samples_split = 2, 
    min_samples_leaf = 1, 
    min_weight_fraction_leaf = 0.0, 
    max_features = 'auto', 
    max_leaf_nodes = None, 
    min_impurity_decrease = 0.0, 
    min_impurity_split = None, 
    ccp_alpha = 0.0, 
    random_state = 42, 
    bootstrap = True, 
    oob_score = True, 
    max_samples = None,
    warm_start = False, 
    n_jobs = -1, 
    verbose = 0, 
)


calibrated_classifier = CalibratedClassifierCV(
    base_estimator = classifier,
    method = 'sigmoid', # 'sigmoid' or 'isotonic'
    ensemble = True,
    cv = 5, # int of 'prefit'
    n_jobs = -1,
)

In [ ]:
%time classifier.fit(X_small, y_cat_5_small)

print(classifier.score(X_small, y_cat_5_small)) # classification accuracy on training set
print(classifier.score(X_valid, y_cat_5_valid)) # classification accuracy on validation set

In [ ]:
%time calibrated_classifier.fit(X_small, y_cat_5_small)

print(calibrated_classifier.score(X_small, y_cat_5_small)) # classification accuracy on training set
print(calibrated_classifier.score(X_valid, y_cat_5_valid)) # classification accuracy on validation set

#### Evaluation

We remind that intuitively, precision is the ability of a binary classifier not to label as positive a sample that is negative, and recall is the ability of the classifier to find all the positive samples.

Evaluation of a multiclass classifier can, in addition to computing the overall accuracy, either be computed using  `macro` average or `weighted` macro-average of precision, recall and F1-score :

- macro : the macro-average of the recall score is obtained by taking the average, over all classes $c$, of the ratio of well-classified points among points with true class $c$, whereas the precision score is obtained by taking the average, over all classes $c$, of the ratio of well-classified points among points with predicted class $c$. The macro-averaged F1-score is taken by applying the F1-score formula to the macro-averages of precision and recall scores.


- weighted macro-average : Same as macro, but averages over classes are ponderated by the ratio of points belonging to each class. This permits to take into account the dataset imbalance over the different classes : scores computed over well-represented classes have more weight on the resulting scores, whereas poorly-represented classes have little impact on the resulting scores. This gives a more realistic evaluation since future point sets to classify will typically follow the same imbalance as observed during evaluation.


For _multi-label_ classification there is also the notion of `micro` average as alternative to macro and weighted-macro averages.

Weighted macro-averages :

In [ ]:
y_predict_valid = classifier.predict(X_valid)

average = 'weighted'

acc = metrics.accuracy_score(y_cat_5_valid, y_predict_valid)
rec = metrics.recall_score(y_cat_5_valid, y_predict_valid, average = average)
prc = metrics.precision_score(y_cat_5_valid, y_predict_valid, average = average)
f1  = metrics.f1_score(y_cat_5_valid, y_predict_valid, average = average)

print('Accuracy : {:.2f}%'.format(acc*100))
print('Recall : {:.2f}%'.format(rec*100))
print('Precision : {:.2f}%'.format(prc*100))
print('F1-score : {:.2f}%'.format(f1*100))

In [ ]:
y_predict_valid = calibrated_classifier.predict(X_valid)

average = 'weighted'

acc = metrics.accuracy_score(y_cat_5_valid, y_predict_valid)
rec = metrics.recall_score(y_cat_5_valid, y_predict_valid, average = average)
prc = metrics.precision_score(y_cat_5_valid, y_predict_valid, average = average)
f1  = metrics.f1_score(y_cat_5_valid, y_predict_valid, average = average)

print('Accuracy : {:.2f}%'.format(acc*100))
print('Recall : {:.2f}%'.format(rec*100))
print('Precision : {:.2f}%'.format(prc*100))
print('F1-score : {:.2f}%'.format(f1*100))

The different scores are better summarized in the following report :

In [ ]:
y_predict_valid = classifier.predict(X_valid)

report = metrics.classification_report(
    y_cat_5_valid, 
    y_predict_valid, 
    target_names = [str(interval) for interval in label_encoder.classes_.tolist()],
)
print(report)

In [ ]:
y_predict_valid = calibrated_classifier.predict(X_valid)

report = metrics.classification_report(
    y_cat_5_valid, 
    y_predict_valid, 
    target_names = [str(interval) for interval in label_encoder.classes_.tolist()],
)
print(report)

#### Confusion matrix

Ultimately, classification preformances are better understood though the _confusion matrix_ of the model on the evaluation set:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

metrics.plot_confusion_matrix(
    classifier, 
    X_valid, 
    y_cat_5_valid, 
    display_labels = [str(interval) for interval in label_encoder.classes_.tolist()],
    include_values = True, 
    xticks_rotation = 45, 
    values_format = None, 
    cmap = 'viridis', 
    ax = ax,
    colorbar = True,
)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

metrics.plot_confusion_matrix(
    calibrated_classifier, 
    X_valid, 
    y_cat_5_valid, 
    display_labels = [str(interval) for interval in label_encoder.classes_.tolist()],
    include_values = True, 
    xticks_rotation = 45, 
    values_format = None, 
    cmap = 'viridis', 
    ax = ax,
    colorbar = True,
)

# 5. Model Interpretation

[Back to top](#plan)

<a id='RF_feature_importance'></a>

### $\bullet$ Random Forest feature importance

[Back to top](#plan)

It's normally not enough to just know that a model can make accurate predictions - we also want to know *how* it's making predictions. The most important way to see this is with *feature importance*.

Scikit-learn's implementation of Random Forests carry _impurity-based_ feature importance scores (the higher, the more important the feature) in the attribute `model.feature_importances_` of a fitted model. The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature on the training set. 

Since they are computed on the training set these scores should be taken with caution, as the importances can be high even for features that are not predictive of the target variable, as long as the model has the capacity to use them to overfit. As a result, features that are deemed of low importance for a bad model (high training score but low validation/cross-validation score) could be very important for a good model. Therefore it is always important to evaluate the predictive power of a model using a validation set (or better with cross-validation) prior to computing importances. These feature importance does not reflect the intrinsic predictive value of a feature by itself, but how important this feature is for a particular random forest.


See more on the [scikit-learn documentation](https://scikit-learn.org/stable/modules/partial_dependence.html).

In [ ]:
def rf_feat_importance(model, df):
    feat_importance = pd.DataFrame({
        'cols': df.columns, 
        'imp': model.feature_importances_,
    })
    feat_importance = feat_importance.sort_values('imp', ascending = False)
    return feat_importance


def rf_unfolded_feat_importance(model, df):
    feat_importance = pd.DataFrame({
        'cols': df.columns, 
        'imp': model.feature_importances_,
    })
    for i, tree in enumerate(model.estimators_):
        feat_importance['imp_' + str(i)] = tree.feature_importances_
    feat_importance = feat_importance.sort_values('imp', ascending = False)
    return feat_importance


# found at
# https://stackoverflow.com/questions/49170296/scikit-learn-feature-importance-calculation-in-decision-trees
def tree_feature_importance(model, normalize = True):

    left_c = model.tree_.children_left
    right_c = model.tree_.children_right

    impurity = model.tree_.impurity    
    node_samples = model.tree_.weighted_n_node_samples 

    # Initialize the feature importance, those not used remain zero
    feature_importance = np.zeros((model.tree_.n_features,))

    for idx, node in enumerate(model.tree_.feature):
        if node >= 0:
            # Accumulate the feature importance over all the nodes where it's used
            feature_importance[node] += (
                impurity[idx]*node_samples[idx] -
                impurity[left_c[idx]]*node_samples[left_c[idx]] -
                impurity[right_c[idx]]*node_samples[right_c[idx]])

    # Number of samples at the root node
    feature_importance /= node_samples[0]

    if normalize:
        normalizer = feature_importance.sum()
        if normalizer > 0:
            feature_importance /= normalizer

    return feature_importance

In [ ]:
best_model.get_params

In [ ]:
model = RandomForestRegressor(
    max_features=0.75, 
    max_samples=0.75, 
    max_depth=10,
    n_estimators=30, 
    n_jobs = -1,  
    random_state=42,
    bootstrap = True,
    oob_score = True,
)

%time model.fit(X_small, y_small)
print_score(model, X_small, y_small, X_valid, y_valid)

In [ ]:
model.feature_importances_

#### Feature importance of a single tree

In [ ]:
# pick the first tree of the forest
tree = model.estimators_[0]
tree.feature_importances_

In [ ]:
feat_importance = rf_feat_importance(tree, X_small)
feat_importance[:20]

In [ ]:
df['All_Artists_pop_sum'].value_counts()

In [ ]:
barh = feat_importance[:30].plot('cols', 'imp', 'barh', figsize = (12,10), legend = False)
barh = barh.invert_yaxis()
barh

The total popularity of artists seems to be the most important feature by a lot, followed by temporal features.

#### Feature importance of the whole random forest

Each tree of the random forest carries its own feature importance along the explanatory variables (variables unseen during training simply have score 0.0). The overall feature importance of the random forest is then naturally defined as the _average, feature by feature, of the importance scores computed by each tree_.

In [ ]:
feat_importance = rf_feat_importance(model, X_small)
feat_importance[:20]

In [ ]:
barh = feat_importance[:33].plot('cols', 'imp', 'barh', figsize = (12,10), legend = False)
barh = barh.invert_yaxis()
barh

When looking at the whole forrest, the main columns are still the same (popularity and date), but we can see more musical features at the top of the list.

#### Ablation of least important features

Least important features can be discarded before re-training the random forest, where it yields a slight improvement of the model's validation $R^2$ score :

In [ ]:
to_keep = feat_importance[feat_importance.imp > 0.005].cols
len(to_keep)

In [ ]:
to_keep

In [ ]:
# perform ablation of columns with low importance
df_keep = df[to_keep].copy()

n_total = len(df_keep)
n_valid = 25000  # same as Kaggle's test set size
n_train = n_total - n_valid
n_small = 25000

X_train_keep, X_valid_keep = split_vals(df_keep, n_train)
y_train, y_valid = split_vals(y, n_train)

X_small_keep, _ = split_vals(df_keep, n_small)
y_small, _ = split_vals(y, n_small)

print('Number of small training data points: X = {}, y = {}'.format(X_small.shape, y_small.shape))
print('Number of full training data points: X = {}, y = {}'.format(X_train.shape, y_train.shape))
print('Number of validation data points: X = {}, y = {}'.format(X_valid.shape, y_valid.shape))

In [ ]:
model = RandomForestRegressor(
    max_features=0.75, 
    max_samples=0.75, 
    max_depth=10,
    n_estimators=30, 
    n_jobs = -1,  
    random_state=42,
    bootstrap = True,
    oob_score = True,
)

%time model.fit(X_small_keep, y_small)
print_score(model, X_small_keep, y_small, X_valid_keep, y_valid)

In [ ]:
feat_importance = rf_feat_importance(model, X_small_keep)

barh = feat_importance.plot('cols', 'imp', 'barh', figsize = (12,10), legend = False)
barh = barh.invert_yaxis()
barh

We can see that in fact, very few features are important.

<a id='permutation_feature_importance'></a>

### $\bullet$ Permutation importance

[Back to top](#plan)

Permutation feature importance is a model inspection technique that can be used for any fitted estimator, and is not restricted to Random Forests. This is especially useful for non-linear or opaque estimators.


The permutation feature importance is defined to be the decrease in a model score when a single feature value is randomly shuffled. This procedure breaks the relationship between the feature and the target, thus the drop in the model score is indicative of how much the model depends on the feature. Precisely, the `permutation_importance` function takes as input a fitted model and a validation set $\mathcal{D}$ with $d$ features, and compute the reference $R^2$ score sc of the model on this dataset. It then loops over a predefined number $k$ of repetitions, where during each loop $i$ a set of $d$ copies of the data are created, with in each copy $\mathcal{D}_{i, j}$ having feature $j$ being randomly shuffled (aka the term _permutation_). The $R^2$ score of the model is then computed on each of these corrupted copies of $\mathcal{D}$, yielding a set $\text{sc}_{i, j}$ of scores. The resulting importance score of each feature $j$ is then given by 

$$ \text{sc}_j = \text{sc} - \frac{1}{k}\sum_{i}\text{sc}_{i, j}$$


The importance of a feature is then higher as the model score decreases after corrupting this feature.
 


As for Random Forests impurity-based feature importance, feature importance based on permutation is of value only for good models (not overfitted and with descent cross-validated score), and does not reflect to the intrinsic predictive value of a feature by itself, but how important this feature is for a particular model.


See more on [scikit-learn documentation](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance)

In [ ]:
model = RandomForestRegressor(
    max_features=0.75, 
    max_samples=0.75, 
    max_depth=10,
    n_estimators=30, 
    n_jobs = -1,  
    random_state=42,
    bootstrap = True,
    oob_score = True,
)

%time model.fit(X_small, y_small)
print_score(model, X_small, y_small, X_valid, y_valid)

In [ ]:
len(X_small.columns)

In [ ]:
result = permutation_importance(
    model, 
    X_valid, 
    y_valid, 
    n_repeats = 10,
    random_state = 42, n_jobs = -1,
)
sorted_idx = result.importances_mean.argsort()

In [ ]:
result

In [ ]:
fig, ax = plt.subplots(figsize = (15, 50))
ax.boxplot(
    result.importances[sorted_idx].T,
    vert = False, 
    labels = X_valid.columns[sorted_idx],
)
ax.set_title("Permutation Importances (validation set)")
fig.tight_layout()
plt.show()

#### Ablation of least important features

Least important features can be discarded before re-training the random forest, where it yields a **notable improvement** of the model's validation $R^2$ score :

In [ ]:
bool_filter = result.importances_mean > 0.005
bool_filter

In [ ]:
to_keep = df.columns[bool_filter]

df_keep = df[to_keep].copy()

n_total = len(df_keep)
n_valid = 25000  # same as Kaggle's test set size
n_train = n_total - n_valid
n_small = 25000

X_train_keep, X_valid_keep = split_vals(df_keep, n_train)
y_train, y_valid = split_vals(y, n_train)

X_small_keep, _ = split_vals(df_keep, n_small)
y_small, _ = split_vals(y, n_small)

print('Number of small training data points: X = {}, y = {}'.format(X_small.shape, y_small.shape))
print('Number of full training data points: X = {}, y = {}'.format(X_train.shape, y_train.shape))
print('Number of validation data points: X = {}, y = {}'.format(X_valid.shape, y_valid.shape))

In [ ]:
len(to_keep)

In [ ]:
model = RandomForestRegressor(
    max_features=0.75, 
    max_samples=0.75, 
    max_depth=10,
    n_estimators=30, 
    n_jobs = -1,  
    random_state=42,
    bootstrap = True,
    oob_score = True,
)

%time model.fit(X_small_keep, y_small)
print_score(model, X_small_keep, y_small, X_valid_keep, y_valid)

In [ ]:
result = permutation_importance(
    model, 
    X_valid_keep, 
    y_valid, 
    n_repeats = 10,
    random_state = 42, n_jobs = -1,
)
sorted_idx = result.importances_mean.argsort()

# plot boxplot
fig, ax = plt.subplots(figsize = (15, 20))
ax.boxplot(
    result.importances[sorted_idx].T,
    vert = False, 
    labels = X_valid.columns[sorted_idx],
)
ax.set_title("Permutation Importances (validation set)")
fig.tight_layout()
plt.show()

#### Ablation after training on full data

Such ablation may hurt performance when the training (and validation) data is abundent :

In [ ]:
model = RandomForestRegressor(
    max_features=0.75, 
    max_samples=0.75, 
    max_depth=10,
    n_estimators=30, 
    n_jobs = -1,  
    random_state=42,
    bootstrap = True,
    oob_score = True,
)

%time model.fit(X_train, y_train)
print_score(model, X_train, y_train, X_valid, y_valid)

In [ ]:
result = permutation_importance(
    model, 
    X_valid, 
    y_valid, 
    n_repeats = 5,
    random_state = 42, 
    n_jobs = -1,
)
sorted_idx = result.importances_mean.argsort()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 50))
ax.boxplot(
    result.importances[sorted_idx].T,
    vert = False, 
    labels = X_valid.columns[sorted_idx],
)
ax.set_title("Permutation Importances (validation set)")
fig.tight_layout()
plt.show()

In [ ]:
bool_filter = result.importances_mean > 0.005
to_keep = df.columns[bool_filter]

df_keep = df[to_keep].copy()

n_total = len(df_keep)
n_valid = 25000  # same as Kaggle's test set size
n_train = n_total - n_valid
n_small = 25000

X_train_keep, X_valid_keep = split_vals(df_keep, n_train)
y_train, y_valid = split_vals(y, n_train)

X_small_keep, _ = split_vals(df_keep, n_small)
y_small, _ = split_vals(y, n_small)

print('Number of small training data points: X = {}, y = {}'.format(X_small.shape, y_small.shape))
print('Number of full training data points: X = {}, y = {}'.format(X_train.shape, y_train.shape))
print('Number of validation data points: X = {}, y = {}'.format(X_valid.shape, y_valid.shape))

In [ ]:
len(to_keep)

In [ ]:
model = RandomForestRegressor(
    max_features=0.75, 
    max_samples=0.75, 
    max_depth=10,
    n_estimators=30, 
    n_jobs = -1,  
    random_state=42,
    bootstrap = True,
    oob_score = True,
)

%time model.fit(X_train_keep, y_train)
print_score(model, X_train_keep, y_train, X_valid_keep, y_valid)

In [ ]:
# default is cv = 5
scores = cross_val_score(model, X_small_keep, y_small, cv = 5, n_jobs = -1)

print(scores)
print("r2 is {:.3f} with a standard deviation of {:.3f}".format(scores.mean(), scores.std()))

<a id='redundant_features'></a>

### $\bullet$ Feature correlations

[Back to top](#plan)

If one computes permutation importance of features when some collinearity occurs, corrupting one feature will have little effect on the models performance because it can get the same information from a correlated feature. Because of that, the permutation importance will show that none of the features are important, which is in contradiction with a high validation that shows that some feature must be important. One approach to handling multicollinearity is by performing hierarchical clustering on the features’ Spearman rank-order correlations, picking a threshold, and keeping a single feature from each cluster.

In [ ]:
# something is weird with this column
cols = df.columns.tolist()
cols.index('saleIs_year_start')

In [ ]:
# something is weird with this column
cols = df.columns.tolist()
cols.remove('saleIs_year_start')
cols

We compute the Spearman rank-order correlation matrix of the explainatory variables using the `scipy` package. Each coefficient is a measure of the monotonicity of the relationship between two columns : it varies between -1 and +1, with 0 implying no correlation and correlations of -1 or +1 implying an exact monotonic relationship. Positive correlations imply that as x increases, so does y, whereas negative correlations imply that as x increases, y decreases.

In [ ]:
len(df.columns)

In [ ]:
corr = np.round(scipy.stats.spearmanr(df[cols]).correlation, 4)
corr.size

In [ ]:
corr.shape

In [ ]:
col_small = [
    'Transmission',
    'Turbocharged',
    'Blade_Extension',
    'Blade_Width',
    'Enclosure_Type',
    'Engine_Horsepower',
    'Hydraulics',
    'Pushblock',
]
corr_small = np.round(scipy.stats.spearmanr(df[col_small]).correlation, 4)
corr_small.size

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
cax = ax.matshow(corr_small)

fig.colorbar(cax)

ax.set_xticklabels([''] + col_small)
ax.set_yticklabels([''] + col_small)


In [ ]:
# corr with size = (65, 65) -> corr_condensed with size (65 * 64)/2
corr_condensed = hierarchy.distance.squareform(1 - corr)
corr_condensed.size

In [ ]:
(65 * 64)/2

In [ ]:
corr_linkage = hierarchy.linkage(corr_condensed, method = 'average')

In [ ]:
corr_linkage

In [ ]:
# compute plot
fig = plt.figure(figsize = (16,20))
dendrogram = hierarchy.dendrogram(
    corr_linkage, 
    labels = cols, 
    orientation = 'left', 
    leaf_font_size = 16,
)
plt.show()

In [ ]:
from collections import defaultdict

In [ ]:
# compute hierarchical clustering
threshold = 0.1
cluster_ids = hierarchy.fcluster(corr_linkage, threshold, criterion = 'distance')

In [ ]:
cluster_ids

In [ ]:
# reshape clustering as {cluster_index : feature_index}
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)

In [ ]:
cluster_id_to_feature_ids

In [ ]:
# keep only first feature for each cluster
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

# shift feature index after 62
selected_features = [(i if i < 62 else i+1) for i in selected_features]

In [ ]:
selected_features

In [ ]:
# retained set of features
selected_features = [df.columns[i] for i in selected_features]
selected_features

In [ ]:
len(selected_features)

#### Perform training on ablated set of features

In [ ]:
df_keep = df[selected_features].copy()

n_total = len(df_keep)
n_valid = 12000  # same as Kaggle's test set size
n_train = n_total - n_valid
n_small = 20000

X_train_keep, X_valid_keep = split_vals(df_keep, n_train)
y_train, y_valid = split_vals(y, n_train)

X_small_keep, _ = split_vals(df_keep, n_small)
y_small, _ = split_vals(y, n_small)

print('Number of small training data points: X = {}, y = {}'.format(X_small.shape, y_small.shape))
print('Number of full training data points: X = {}, y = {}'.format(X_train.shape, y_train.shape))
print('Number of validation data points: X = {}, y = {}'.format(X_valid.shape, y_valid.shape))

In [ ]:
model = RandomForestRegressor(
    n_estimators = 50,
    max_depth = 15,
    min_samples_split = 10,
    max_features = 0.5, # default = 'auto'/None
    n_jobs = -1, 
    random_state = 42,
    
    bootstrap = True,
    oob_score = True, # default = False, 
    max_samples = 0.75, # default = None,
)

%time model.fit(X_train_keep, y_train)
print_score(model, X_train_keep, y_train, X_valid_keep, y_valid)

The model performance does not improve. Let us now see if some features shows up as important and wasn't before due to the correlation issues :

#### Permutation of feature importance after feature selection

In [ ]:
result = permutation_importance(
    model, 
    X_valid_keep, 
    y_valid, 
    n_repeats = 5,
    random_state = 42, 
    n_jobs = -1,
)
sorted_idx = result.importances_mean.argsort()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 50))
ax.boxplot(
    result.importances[sorted_idx].T,
    vert = False, 
    labels = X_valid.columns[sorted_idx],
)
ax.set_title("Permutation Importances (validation set)")
fig.tight_layout()
plt.show()

**Conclusion** : We now see that `Blade_Extension`overtakes `YearMade`as the most discriminant variable. This makes a lot of sense since according to the previous dendrogram this features is extremely correlated with the feature `Engine_Horsepower`, which indeed must heavily impact the sale price.

<a id='feature_explicability'></a>

### $\bullet$ Individual feature explicability

[Back to top](#plan)

installs requiered:
- `pip install plotnine`
- `conda install matplotlib=2.2.3`

#### Partial Dependance Plots of explanatory variables

In [ ]:
sample = get_sample(df_raw[df_raw.YearMade > 1930], 500)

In [ ]:
help(stat_smooth)

In [ ]:
plot = ggplot(sample, aes('YearMade', 'SalePrice')) + stat_smooth(se = True, method = 'auto')
plot

#### Individual Conditional Expectation plots of explanatory variables


Individual Conditional Expectation (ICE) plots display one line per instance that shows how the instance's prediction changes when a feature changes. The values for a line (and one instance) can be computed by keeping all other features the same, creating variants of this instance by replacing the feature's value with values from a grid and making predictions with the black box model for these newly created instances. The result is a set of points for an instance with the feature value from the grid and the respective predictions. An ICE plot is more precise than a partial dependence plot as the latter does not focus on specific instances, but on an overall average, which can obscure a heterogeneous relationship created by interactions.

See more on ICE plots [here](https://christophm.github.io/interpretable-ml-book/ice.html)

In [ ]:
# x = get_sample(X_train[X_train.YearMade > 1930], 500)
x = get_sample(df_keep[df_keep.YearMade > 1930], 500)

In [ ]:
def plot_pdp(model, x, feat, clusters = None, feat_name = None):
    feat_name = feat_name or feat
    p = pdp.pdp_isolate(model, x, feat)
    return pdp.pdp_plot(p, feat_name, plot_lines=True,
                        cluster=clusters is not None,
                        n_cluster_centers=clusters)

In [ ]:
plot_pdp(model, x, 'YearMade')

In [ ]:
# cluster 500 lines into 5 clusters
plot_pdp(model, x, 'YearMade', clusters = 5)

In [ ]:
x.columns

In [ ]:
feats = ['saleYear', 'YearMade']
p = pdp.pdp_interact(model, x, feats)
pdp.pdp_interact_plot(p, feats)
# handle error of gdp_plot updates: 
# https://forums.fast.ai/t/pbpbox-error-on-pdp-interact-plot/28468/2
# in C:\Users\Jb\.conda\envs\fastai-cpu-v0.7\Lib\site-packages\pdpbox/pdp.py
# Line 1147 change 
# ax.clabel(c2, contour_label_fontsize=9, inline=1) 
# to ax.clabel(c2, fontsize=9, inline=1)

[Back to top](#plan)